In [1]:
from gensim import corpora
from gensim.models import LdaModel
import json
from data_analysis.OpeningLDA import OpeningLDA
from classes.QueryHandler import QueryHandler

In [ ]:
opening_lda = OpeningLDA(num_passes=5, print_debug=True) 

print(len(opening_lda.texts.keys()))
opening_lda.lda_model.print_topics(num_words=15)

opening_lda.save('lda')


In [2]:
opening_lda = OpeningLDA.load("lda")

Getting texts
New len(self.texts): 483
10027


In [3]:
opening_lda.__construct_topic_doc_matrix__()

Processing "B44" (0/483)
Processing "D12" (1/483)
Processing "B88" (2/483)
Processing "D15" (3/483)
Processing "B43" (4/483)
Processing "B81" (5/483)
Processing "B75" (6/483)
Processing "D23" (7/483)
Processing "D24" (8/483)
Processing "B72" (9/483)
Processing "B86" (10/483)
Processing "D70" (11/483)
Processing "B26" (12/483)
Processing "D84" (13/483)
Processing "B19" (14/483)
Processing "D83" (15/483)
Processing "B21" (16/483)
Processing "D48" (17/483)
Processing "D41" (18/483)
Processing "B17" (19/483)
Processing "B28" (20/483)
Processing "B10" (21/483)
Processing "D46" (22/483)
Processing "B87" (23/483)
Processing "D25" (24/483)
Processing "B73" (25/483)
Processing "B74" (26/483)
Processing "D22" (27/483)
Processing "B80" (28/483)
Processing "B89" (29/483)
Processing "D14" (30/483)
Processing "B42" (31/483)
Processing "B45" (32/483)
Processing "D13" (33/483)
Processing "B11" (34/483)
Processing "D47" (35/483)
Processing "D40" (36/483)
Processing "B16" (37/483)
Processing "B29" (38/4

In [4]:
query_handler = QueryHandler()
query_handler.opening_lda = opening_lda

sample_query1 = { 
                 "message": "capture the center, develop light square bishop",
                 "color": "white" 
}
query_handler.handle_user_query(sample_query1, debug=True)


Relative index matches:
C21 : 0.3650990099009901
C22 : 0.3564356435643564
C44 : 0.2504950495049505
D43 : 0.23335128343205888
D46 : 0.23263327948303716
D48 : 0.23263327948303716
D45 : 0.23263327948303716
D47 : 0.23263327948303716
D10 : 0.2315270935960591
E11 : 0.22537431048069345
C20 : 0.22514910536779323
C23 : 0.22266401590457258
C24 : 0.22266401590457258
C28 : 0.22266401590457258
D11 : 0.22212270488132557
D17 : 0.22212270488132557
D12 : 0.22212270488132557
D15 : 0.22212270488132557
D14 : 0.22212270488132557
D18 : 0.22212270488132557
D16 : 0.22212270488132557
D13 : 0.22212270488132557
D49 : 0.21809369951534732
C26 : 0.21669980119284293
C27 : 0.21669980119284293
D19 : 0.21540528437080161
A51 : 0.20565014743650717
A52 : 0.20545990678207934
E12 : 0.20473251028806586
E17 : 0.19753086419753085
E14 : 0.19753086419753085
E16 : 0.19753086419753085
E15 : 0.19753086419753085
E18 : 0.19753086419753085
E19 : 0.18518518518518517
E13 : 0.18518518518518517
C00 : 0.1732490784623486
C15 : 0.1718448306